# Table operations using `Spark SQL`, `Hive context`, and `Presto`

#### Topics covered in this example
* Creating an external table using `%%sql` magic and querying the table.
* Querying the table using a hive context.
* Connecting to the table using a Presto connector and querying the table.

***

## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* The EMR cluster attached to this notebook should have the `Spark`, `Hive` and `Presto` applications installed.
* This example uses a public dataset from s3, hence the EMR cluster attached to this notebook must have internet connectivity.
* This notebook uses the `PySpark` kernel.
***

## Introduction
In this example, we are going to create an external Hive table and query the table using `spark sql magic`, `hive context` and `presto`.

We use the <a href="https://s3.amazonaws.com/amazon-reviews-pds/readme.html" target="_blank">Amazon customer review dataset</a> that is publically accessible in s3.  
This dataset is a collection of reviews written in the Amazon.com marketplace and associated metadata from 1995 until 2015. This is intended to facilitate study into the properties (and the evolution) of customer reviews potentially including how people evaluate and express their experiences with respect to products at scale.
***

## Sql magic example

<a href="https://ipython.readthedocs.io/en/stable/interactive/magics.html" target="_blank">Magic commands</a> are pre-defined functions(`magics`) in Jupyter kernel that execute the supplied commands.  
Sql magic extension makes it possible to write SQL queries directly into code cells.  
For more information about these magic commands, see the <a href="https://github.com/jupyter-incubator/sparkmagic" target="_blank">GitHub repo</a>.


You can see all of the available magics with the help of `%lsmagic`.

In [ ]:
%lsmagic

Create a table `Books` from the Amazon customer reviews data for books using the sql magic `%%sql`.

`%%sql` marks an entire cell as a SQL block which allows us to enter multi-line SQL statements.

In [ ]:
%%sql
CREATE EXTERNAL TABLE IF NOT EXISTS Books(review_id STRING,product_title STRING,star_rating INT,verified_purchase STRING,review_date DATE,year INT)
STORED AS PARQUET LOCATION "s3://amazon-reviews-pds/parquet/product_category=Books"

Show existing tables.

In [ ]:
%%sql
show tables

Show the details for the table `Books`.

In [ ]:
%%sql
describe formatted Books

Execute a query to find the top 20 best reviewed books ordered by descending `star_ratings` and limited to 20 records.

In [ ]:
%%sql
SELECT product_title, AVG(star_rating), count(review_id) AS review_count FROM Books
WHERE review_date >= "2015-08-28" AND review_date <= "2015-08-30" AND verified_purchase="Y"
GROUP BY product_title
ORDER BY SUM(star_rating) DESC
LIMIT 20

***
## Hive context example

A `Hive context` is an instance of the Spark SQL execution engine that integrates with data stored in Hive.  
The following example shows how to query the table `Books` using the hive context.

Import dependencies.

In [ ]:
from pyspark.sql import HiveContext

Initiate the hive context and display the list of tables in the default schema.

In [ ]:
sqlContext = HiveContext(sc)
sqlContext.sql("use default")
sqlContext.sql("show tables").show()

Display the sample table records.

In [ ]:
books = sqlContext.table("default.books")
books.show()

Execute a query to count the number of purchases with high customer ratings (ratings greater than or equal to 4).

In [ ]:
sqlContext.sql("Select count(product_title) as count_of_purchases_with_high_rating from books where star_rating >=4").show()

***
## Presto example

Analyze data stored in a database via Presto with the PyHive Presto Python library.

Install `pyhive` and `requests` from the public PyPI repository.

In [ ]:
sc.install_pypi_package("pyhive")
sc.install_pypi_package("requests")

Import dependencies.

In [ ]:
from pyhive import presto

Use the following configuration to connect to the database by using the Presto connector.

`host` : Host name or ip address of the database server.  
`port` : Port of the database server.  
`catalog` : Name of the catalog. A Presto catalog contains schemas and references of a data source via a connector.   
`schema` : Name of the schema.

In [ ]:
cursor = presto.connect(host = "localhost", port = 8889, catalog = "hive", schema = "default").cursor()

List the tables created in the `default` schema.

In [ ]:
cursor.execute("show tables")
results = cursor.fetchall()
print(results)

Query the books table using presto to get the count of `product_title`

In [ ]:
cursor.execute("Select count(product_title) from Books")
results = cursor.fetchall()
print(results)

***
## Cleanup

Delete the table.

In [ ]:
%%sql
DROP TABLE IF EXISTS Books

Lastly, use the `uninstall_package` Pyspark API to uninstall the `pyhive` and `requests` libraries that were installed using the `install_package` API.

In [ ]:
sc.uninstall_package("pyhive")
sc.uninstall_package("requests")